In [2]:
import json
import os
import getpass
from tqdm.auto import tqdm

In [3]:
import getpass
def _get_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

_get_env("GROQ_API_KEY")

In [4]:
with open('/home/nkama/LLM_and_RAG_Course/LLM_and_RAG-/personal_assg_project/data/documents-with-ids.json', 'rt') as f_in:
    documents = json.load(f_in)

In [47]:
documents[1]

{'text': 'GitHub - DataTalksClub data-engineering-zoomcamp#prerequisites',
 'section': 'General course-related questions',
 'question': 'Course - What are the prerequisites for this course?',
 'course': 'data-engineering-zoomcamp',
 'id': '1f6520ca'}

## Implementing Key Word Search with Elasticsearch

In [5]:
from elasticsearch import Elasticsearch
es_client = Elasticsearch('http://localhost:9200')

In [11]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"}
        }
    }
}

index_name = "course-questions"

es_client.indices.create(index=index_name, body=index_settings)

In [12]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

100%|██████████| 948/948 [00:05<00:00, 159.74it/s]


In [13]:
query = 'I just disovered the course. Can I still join it?'

In [14]:
def elastic_search(query):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)

    result_docs = []

    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])

    return result_docs

In [19]:
def build_prompt(query):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

    context = ""
    search_results = elastic_search(query)
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"

    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

prompt = build_prompt(query)
def llm(prompt):
    response = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content":prompt,
            }
        ],
        model="llama3-8b-8192",
    )

    # print the response
    print(response.choices[0].message.content)

query = 'how do I run kafka?'

def keyword_rag(query):
    search_results = elastic_search(query)
    prompt = build_prompt(query)
    answer = llm(prompt)
    return answer

In [29]:
from groq import Groq
client = Groq()

In [30]:
prompt = build_prompt(query)
def llm(prompt):
    response = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content":prompt,
            }
        ],
        model="llama3-8b-8192",
    )

    # print the response
    print(response.choices[0].message.content)


In [55]:
query = 'how do I run kafka?'

def keyword_rag(query):
    search_results = elastic_search(query)
    prompt = build_prompt(query)
    answer = llm(prompt)
    return answer

In [31]:
rag(query)

Based on the provided context, the answer to the question "how do I run kafka?" is:

In the project directory, run:
java -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java


## Implementing Vector Search with Elastic Search

In [ ]:
from elasticsearch import Elasticsearch
es_client = Elasticsearch('http://localhost:9200')

In [6]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"},
            "id": {"type": "keyword"},
            "question_vector": {
                "type": "dense_vector",
                "dims": 384,
                "index": True,
                "similarity": "cosine"
            },
            "text_vector": {
                "type": "dense_vector",
                "dims": 384,
                "index": True,
                "similarity": "cosine"
            },
            "question_text_vector": {
                "type": "dense_vector",
                "dims": 384,
                "index": True,
                "similarity": "cosine"
            },
        }
    }
}

index_name = "course-questions"

es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body=index_settings)


ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [7]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer("all-mpnet-base-v2")

/home/nkama/.pyenv/versions/3.10.6/envs/llmMath/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [14]:
def create_vector_index(es_client, index_name="course-questions"):
    index_settings = {
        "settings": {
            "number_of_shards": 1,
            "number_of_replicas": 0
        },
        "mappings": {
            "properties": {
                "text": {"type": "text"},
                "section": {"type": "text"},
                "question": {"type": "text"},
                "course": {"type": "keyword"},
                "id": {"type": "keyword"},
                "question_text_vector": {
                    "type": "dense_vector",
                    "dims": 384,  # Make sure this matches your vector dimensions
                    "index": True,
                    "similarity": "cosine"
                }
            }
        }
    }

    # Delete the existing index if it exists
    es_client.indices.delete(index=index_name, ignore=[400, 404])

    # Create the new index
    es_client.indices.create(index=index_name, body=index_settings)
    print(f"Vector index '{index_name}' created successfully.")

# Call this function to recreate your index
create_vector_index(es_client)

Vector index 'course-questions' created successfully.


/tmp/ipykernel_46379/3446748621.py:25: DeprecationWarning: Passing transport options in the API method is deprecated. Use 'Elasticsearch.options()' instead.
  es_client.indices.delete(index=index_name, ignore=[400, 404])


In [17]:
from tqdm import tqdm
from elasticsearch.helpers import bulk

def index_documents(es_client, documents, index_name="course-questions", model=None):
    def generate_actions():
        for doc in documents:
            if model:
                # Encode vector fields if a model is provided
                question = doc['question']
                text = doc['text']
                qt = question + ' ' + text
                doc['question_vector'] = model.encode(question).tolist()
                doc['text_vector'] = model.encode(text).tolist()
                doc['question_text_vector'] = model.encode(qt).tolist()

            yield {
                "_index": index_name,
                "_id": doc['id'],
                "_source": doc
            }

    # Use bulk indexing for better performance
    success, failed = bulk(es_client, generate_actions(), stats_only=True, raise_on_error=False)

    print(f"Indexed {success} documents successfully.")
    if failed:
        print(f"Failed to index {failed} documents.")

# Usage
from sentence_transformers import SentenceTransformer

# Load your documents
with open('/home/nkama/LLM_and_RAG_Course/LLM_and_RAG-/personal_assg_project/data/documents-with-ids.json','r') as f:
    documents = json.load(f)

# If you need to encode vector fields
model = SentenceTransformer('all-MiniLM-L6-v2')  # or your preferred model

# Index the documents
index_documents(es_client, documents, model=model)

Indexed 948 documents successfully.


In [13]:
for doc in tqdm(documents):
    try:
        es_client.index(index=index_name, document=doc)
    except Exception as e:
        print(e)

In [10]:
def elastic_search_knn(field, vector, course):
    knn = {
        "field": field,
        "query_vector": vector,
        "k": 5,
        "num_candidates": 10000,
        "filter": {
            "term": {
                "course": course
            }
        }
    }

    search_query = {
        "knn": knn,
        "_source": ["text", "section", "question", "course", "id"]
    }

    es_results = es_client.search(
        index=index_name,
        body=search_query
    )

    result_docs = []

    for hit in es_results['hits']['hits']:
        result_docs.append(hit['_source'])

    return result_docs

In [11]:
search_term = "windows or mac?"
query = 'I just discovered the course. Can I still join it?'
vector_search_term = model.encode(search_term)

In [18]:
elastic_search_knn('text_vector', vector_search_term, 'data-engineering-zoomcamp')

BadRequestError: BadRequestError(400, 'search_phase_execution_exception', 'failed to create query: [knn] queries are only supported on [dense_vector] fields')

In [ ]:
def vector_rag(field, vector, course):
    search_results = elastic_search_knn(field, vector, course)
    prompt = build_prompt(query)
    answer = llm(prompt)
    return answer

In [ ]:
vector_rag('text_vector', vector_search_term, 'data-engineering-zoomcamp')